# Essay 5: 

# A Better Benchmark for Hierarchical Risk Parity (HRP) Portfolios: Part 2

# Table of Contents

    I. Motivation & Summary
    II. HRP Walkthrough
    III. Revisiting the JoPM 10-Asset Example
    Appendix A: Version Control / GitHub
    Appendix B: Quick Comment on How to do Research
    Appendix C: Revisiting the Appendixes of Essay 1
    Appendix D: Solving Mystery 1
    Appendix E: Solving Mystery 2


# I.	Motivation & Summary

![https://i.imgur.com/YLWlLbW.png](https://i.imgur.com/YLWlLbW.png "is_this_ML_title")

# II. HRP Walkthrough

This is a continuation of <a href="https://kyleessays.substack.com/p/essay-1-a-better-benchmark-for-hierarchical">Essay 1</a>, which is a critique of <a href="https://jpm.pm-research.com/content/42/4/59">Building Diversified Portfolios that Outperform Out-of-Sample</a> (BDPOOS), published by <a href="https://jpm.pm-research.com/quant-of-2019">2019 Quant of the Year Marcos López de Prado</a> (MLdP) in the Summer 2016 Issue of The Journal of Portfolio Management (JoPM).

Warning: Like Essay 1, this is mostly a math essay.

Here's a quick summary of how HRP weights are computed:
1.  Convert covariance matrix to a correlation matrix.
2.  Convert the correlation matrix to a distance matrix.
3.  Cluster the distances into groups.
4.  Sort the original covariance matrix based on those groupings.
5a.  Recursively bisect that sorted covariance matrix.
5b.  For each bisection, apply IVP weights to that subset.

In the case of the 10-Asset example from JoPM, the 10 assets are artificially constructed to be correlated to each other: 

![https://i.imgur.com/gIGOG7K.png](https://i.imgur.com/gIGOG7K.png "table_2")

The above assets clearly fall neatly into the following clusters: {9, 2, 10}, {1, 7}, {3, 6}, {4}, and {5, 8}.

The first bisection would carve the above into these two super-sets: {9, 2, 10, 1, 7} and {3, 6, 4, 5, 8}.

But what if we didn't use asset 4, which is uncorrelated to anything else...

Then the first bisection might give us the super-sets {9, 2, 10, 1} and {7, 3, 6, 5, 8}.

Wait what?  Assets 1 & 7 could be separated from each other right at the start?  They have a correlation of 0.97!  You create clusters and then just ignore them at the first step?

Let's run some code to confirm.

# III. Revisiting the JoPM 10-Asset Example

Let's run some code:

In [1]:
import scipy.cluster.hierarchy as sch, numpy as np, pandas as pd
import os
import HRP_v5 as hrpm # modified version to include print statements & be Python3 compatible

In [2]:
# Load data:
cwd = os.getcwd()
csv_dir_01 = cwd + '\\Simulated_Gaussians_Seed12345.csv'

x = pd.read_csv(csv_dir_01, index_col=0)

# Get covariances to be input to HRP:
covariances, corr = x.cov(), x.corr()

# Convert correlations to distances: 
dist = hrpm.correlDist(corr)
link = sch.linkage(dist, 'single')

sortIx = hrpm.getQuasiDiag(link)
sortIx = corr.index[sortIx].tolist()  # recover labels
df0 = corr.loc[sortIx, sortIx]  # reorder

C:\Users\Kyle\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  if sys.path[0] == '':


I've modified MLdP's HRP.py code (to the HRP_v5.py that you imported above, assuming you properly imported the branch essay_05) as follows:

(1) Inserted numerous "print" statements so that we can follow the algorithm step-by-step.

(2) Remove Python 2 code (such as "xrange") so that this can run in this Jupyter Python 3 environment.

Let's call HRP on and observe the first bisection:

In [3]:
# Capital allocation
hrp_weights = hrpm.getRecBipart(covariances, sortIx)

[['9', '2', '10', '1', '7', '3', '6', '4', '5', '8']]
-----
cItems0
['9', '2', '10', '1', '7']
-----
cItems1
['3', '6', '4', '5', '8']
-----
cVar0
0.5253929089614129
-----
cVar1
0.3669878839679833
-----
alpha
0.4112458345985701
-----
cItems0
['9', '2']
-----
cItems1
['10', '1', '7']
-----
cVar0
1.0253128128876061
-----
cVar1
0.5713503792630303
-----
alpha
0.35784026466686814
-----
cItems0
['3', '6']
-----
cItems1
['4', '5', '8']
-----
cVar0
1.0160858695552584
-----
cVar1
0.564640754896351
-----
alpha
0.3572032925631514
-----
cItems0
['9']
-----
cItems1
['2']
-----
cVar0
1.0753466770005407
-----
cVar1
1.0090213488712376
-----
alpha
0.48408982307681414
-----
cItems0
['10']
-----
cItems1
['1', '7']
-----
cVar0
1.0785866856713524
-----
cVar1
1.013014325747844
-----
alpha
0.48432484026219313
-----
cItems0
['3']
-----
cItems1
['6']
-----
cVar0
1.0007393631661812
-----
cVar1
1.0643098248127214
-----
alpha
0.5153919969598297
-----
cItems0
['4']
-----
cItems1
['5', '8']
-----
cVar0
1.0118547250

Observe from the above output that indeed the first cut produces these two super sets:

cItems0
['9', '2', '10', '1', '7']

cItems1
['3', '6', '4', '5', '8']

Now let's observe the same bisection if we removed the uncorrelated asset number 4:

In [4]:
y = pd.read_csv(csv_dir_01, index_col=0)
y = y.iloc[:,[0,1,2,4,5,6,7,8,9]]

# Get covariances to be input to HRP:
covariances, corr = y.cov(), y.corr()

# Convert correlations to distances: 
dist = hrpm.correlDist(corr)
link = sch.linkage(dist, 'single')

sortIx = hrpm.getQuasiDiag(link)
sortIx = corr.index[sortIx].tolist()  # recover labels
df0 = corr.loc[sortIx, sortIx]  # reorder

# Capital allocation
hrp_weights = hrpm.getRecBipart(covariances, sortIx)

C:\Users\Kyle\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix
  if __name__ == '__main__':


[['9', '2', '10', '5', '8', '3', '6', '1', '7']]
-----
cItems0
['9', '2', '10', '5']
-----
cItems1
['8', '3', '6', '1', '7']
-----
cVar0
0.6281019660185938
-----
cVar1
0.37268433108485094
-----
alpha
0.37239152071076864
-----
cItems0
['9', '2']
-----
cItems1
['10', '5']
-----
cVar0
1.0253128128876061
-----
cVar1
0.5165043040829743
-----
alpha
0.33499712670061743
-----
cItems0
['8', '3']
-----
cItems1
['6', '1', '7']
-----
cVar0
0.519191812730093
-----
cVar1
0.5763578525427326
-----
alpha
0.5260901178763099
-----
cItems0
['9']
-----
cItems1
['2']
-----
cVar0
1.0753466770005407
-----
cVar1
1.0090213488712376
-----
alpha
0.48408982307681414
-----
cItems0
['10']
-----
cItems1
['5']
-----
cVar0
1.0785866856713524
-----
cVar1
1.0058358778581784
-----
alpha
0.48254893007635025
-----
cItems0
['8']
-----
cItems1
['3']
-----
cVar0
1.0748331551216201
-----
cVar1
1.000739363166181
-----
alpha
0.48215099898880887
-----
cItems0
['6']
-----
cItems1
['1', '7']
-----
cVar0
1.0643098248127214
-----
cVar

The first cut produced these two super sets:

cItems0 ['9', '2', '10', '5']

cItems1 ['8', '3', '6', '1', '7']


Indeed, the cluster {5, 8} is destroyed before we even get to take advantage of its status as a cluster.  

That {5, 8} appears before {1, 7} in the example without {4} is a property of the scipy linkage procedure; you can add additional print statements to confirm how this happens.

Can this process be made better?  Of course.  That's another essay.  

And we didn't even address the silly choice of HRP to apply IVP weights at each recursion, instead of, say, ERC weights.  That is, actually account for correlations that JoPM was so hasty to insist were excluded by their straw-man IVP benchmark.

# Appendix A: Version Control / GitHub

Any code, data, and other artifacts produced for this essay will be saved at https://github.com/kyle-binder-essays/essay_repo1/tree/essay_05. 

# Appendix B: Quick Comment on How to do Research

Jupyter is well suited for presentation of results.  But if you’re not sure what your research results will be and need to run lots of scenarios quickly, a more optimal workflow is to simply open multiple Spyder consoles and test as many variations of code as you desire.  

However, once you need to share and communicate results with a team, the above approach fails to scale.

Is there a more optimal balance between doing research and sharing research results more quickly?  At first glance, NBDEV seems promising (and I’m optimistic given previous successes of the fastai team).

# Appendix C: Revisiting the Appendixes of Essay 1

The Appendixes of Essay 1 explored simple examples to better understand the Hierarchical Risk Parity (HRP) and Equal Risk Contribution (ERC) algorithms.  The simple examples presented us with two unintuitive results:

**Mystery #1**: 

HRP can assign very different weights for the same asset when it appears twice in the input variables.  

**Mystery #2**: 

HRP output weights can be very sensitive to small perturbations of inputs.

These are unintuitive results, and most finance practitioners would tell you they’re UNDESIRABLE results.

**Mystery #1 Details**: 


In Figure 5 from <a href="https://kyleessays.substack.com/p/essay-1-a-better-benchmark-for-hierarchical">Essay 1</a>, why does B get much larger weight than A and D?  A, B, and D are the same asset (each has variance = 1.0, and all three of {A,B,D} are 100% correlated to each other).  ERC assigns {A,B,D} the expected behavior of equal weights.

Here are Figures 5 and 6 from Essay #1; { X1, X2, … X5} are from the original BDPOOS data set, normalized to have variance = 1.0:


![https://i.imgur.com/zK3Ya3h.png](https://i.imgur.com/zK3Ya3h.png "fig5")

![https://i.imgur.com/yzEzlc0.png](https://i.imgur.com/yzEzlc0.png "fig6")


**Mystery #2 Details**: 


In Figure 6 from Essay #1, when we decrease epsilon from 1 to an arbitrarily small positive number (but not zero), the HRP weights converge on a desirable property: HRP allocates equally between the cluster {A,B,D} and the cluster {C,E}.

However, when we drop epsilon from 0.01 to 0, there is a large shift in HRP weights (but not ERC weights).  

More problematic: HRP no longer allocates equally between the cluster {A,B,D} and the cluster {C,E}; that is, the sum of the weights of A+B+D is not the same as the sum of the weights of C+E.

Again, ERC assigns {A,B,D} the expected behavior of (near) equal weights for both epsilon=0 and epsilon=0.1.

So, how do these things happen?


# Appendix D: Solving Mystery 1

Let's run some code:

In [5]:
import scipy.cluster.hierarchy as sch, numpy as np, pandas as pd
import os
import HRP_v5 as hrpm

In [6]:
# Load data:
cwd = os.getcwd()
csv_dir_01 = cwd + '\\Simulated_Gaussians_Seed12345.csv'

x_all = pd.read_csv(csv_dir_01, index_col=0)


Next let's load the data set corresponding to epsilon = 0.1:

In [7]:
# Iterate over several values of epsilon and collect all outputs into single dataframe:
iterator_array = pd.DataFrame(range(0,20,5))
# Initialize:
figure6 = pd.DataFrame(columns=list(range(10,11,10)), data=None, \
                               index=['epsilon'])
for iterator in figure6.columns:

    # Ugh, don't need to specify float in Python 3+:
    epsilon = float(iterator) / float(100)

    # Define columns {A,B,C,D,E}:
    x_A = pd.DataFrame(index=x_all.index, data=x_all.iloc[:, 2].values, columns=['x_A'])
    x_B = pd.DataFrame(index=x_all.index, data=( \
                (1 - epsilon) * x_all.iloc[:, 2].values + \
                (epsilon * x_all.iloc[:, 4].values) \
        ), columns=['x_B'])
    x_D = pd.DataFrame(index=x_all.index, data=( \
                (1 - epsilon) * x_all.iloc[:, 2].values + \
                (epsilon * x_all.iloc[:, 0].values) \
        ), columns=['x_D'])
    x_C = pd.DataFrame(index=x_all.index, data=x_all.iloc[:, 3].values, columns=['x_C'])
    x_E = pd.DataFrame(index=x_all.index, data=( \
                (1 - epsilon) * x_all.iloc[:, 3].values + \
                (epsilon * x_all.iloc[:, 1].values) \
        ), columns=['x_E'])

    # Normalize all columns to have variance of 1:
    x_Az = x_A / x_A.std()
    x_Bz = x_B / x_B.std()
    x_Cz = x_C / x_C.std()
    x_Dz = x_D / x_D.std()
    x_Ez = x_E / x_E.std()

    x_concat = [x_Az, x_Bz, x_Cz, x_Dz, x_Ez]
    x = pd.concat(x_concat, axis=1)

    # Get covariance to be input to HRP:
    covariances, corr = x.cov(), x.corr()

    # Get HRP Weights:
    dist = hrpm.correlDist(corr)
    link = sch.linkage(dist, 'single')
    sortIx = hrpm.getQuasiDiag(link)
    sortIx = corr.index[sortIx].tolist()  # recover labels
    df0 = corr.loc[sortIx, sortIx]  # reorder

    # Capital allocation of HRP weights:
    # B is in the lead here - why does it get highest weight...
    hrp_s = hrpm.getRecBipart(covariances, sortIx)
    hrp_df = pd.DataFrame(index=hrp_s.index, data=hrp_s.values, columns=['wts'])


[['x_C', 'x_E', 'x_B', 'x_A', 'x_D']]
-----
cItems0
['x_C', 'x_E']
-----
cItems1
['x_B', 'x_A', 'x_D']
-----
cVar0
0.9969553731227971
-----
cVar1
0.9945518529188007
-----
alpha
0.49939655749862033
-----
cItems0
['x_C']
-----
cItems1
['x_E']
-----
cVar0
0.9999999999999964
-----
cVar1
0.9999999999999929
-----
alpha
0.4999999999999991
-----
cItems0
['x_B']
-----
cItems1
['x_A', 'x_D']
-----
cVar0
1.0
-----
cVar1
0.9969491877753843
-----
alpha
0.49923613173452497
-----
cItems0
['x_A']
-----
cItems1
['x_D']
-----
cVar0
1.0000000000000013
-----
cVar1
1.0
-----
alpha
0.49999999999999967
-----


C:\Users\Kyle\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix


What happened?

Well, at the first bisection, we have the 5 assets separated as {C, E} and {B, A, D}.  So far, so good.  C+E are correlated to each other, and {A, B, D} are correlated.  And since all 5 assets have variance 1.00, an IVP allocation will assign 50% capital to the {C, E} superset and 50% capital to the {B, A, D} superset.

However, at the next step of the recursion, we separate {B, A, D} into {B} + {A, D}.  Again, each asset has variance 1.00, and an IVP allocation will assign 50% capital to the {B} set and 50% capital to the {A, D} set.

That's why {B} gets twice the weight of {A} and {D}, even though the three {A, B, D} are essentially the same asset, just slightly perturbed.

# Appendix E: Solving Mystery 2

Now let's load the data set corresponding to epsilon = 0.0:

In [8]:
# Iterate over several values of epsilon and collect all outputs into single dataframe:
iterator_array = pd.DataFrame(range(0,20,5))
# Initialize:
figure6 = pd.DataFrame(columns=list(range(0,1,10)), data=None, \
                               index=['epsilon'])
for iterator in figure6.columns:

    # Ugh, don't need to specify float in Python 3+:
    epsilon = float(iterator) / float(100)

    # Define columns {A,B,C,D,E}:
    x_A = pd.DataFrame(index=x_all.index, data=x_all.iloc[:, 2].values, columns=['x_A'])
    x_B = pd.DataFrame(index=x_all.index, data=( \
                (1 - epsilon) * x_all.iloc[:, 2].values + \
                (epsilon * x_all.iloc[:, 4].values) \
        ), columns=['x_B'])
    x_D = pd.DataFrame(index=x_all.index, data=( \
                (1 - epsilon) * x_all.iloc[:, 2].values + \
                (epsilon * x_all.iloc[:, 0].values) \
        ), columns=['x_D'])
    x_C = pd.DataFrame(index=x_all.index, data=x_all.iloc[:, 3].values, columns=['x_C'])
    x_E = pd.DataFrame(index=x_all.index, data=( \
                (1 - epsilon) * x_all.iloc[:, 3].values + \
                (epsilon * x_all.iloc[:, 1].values) \
        ), columns=['x_E'])

    # Normalize all columns to have variance of 1:
    x_Az = x_A / x_A.std()
    x_Bz = x_B / x_B.std()
    x_Cz = x_C / x_C.std()
    x_Dz = x_D / x_D.std()
    x_Ez = x_E / x_E.std()

    x_concat = [x_Az, x_Bz, x_Cz, x_Dz, x_Ez]
    x = pd.concat(x_concat, axis=1)

    # Get covariance to be input to HRP:
    covariances, corr = x.cov(), x.corr()

    # Get HRP Weights:
    dist = hrpm.correlDist(corr)
    link = sch.linkage(dist, 'single')
    sortIx = hrpm.getQuasiDiag(link)
    sortIx = corr.index[sortIx].tolist()  # recover labels
    df0 = corr.loc[sortIx, sortIx]  # reorder

    # Capital allocation of HRP weights:
    # B is in the lead here - why does it get highest weight...
    hrp_s = hrpm.getRecBipart(covariances, sortIx)
    hrp_df = pd.DataFrame(index=hrp_s.index, data=hrp_s.values, columns=['wts'])

[['x_D', 'x_A', 'x_B', 'x_C', 'x_E']]
-----
cItems0
['x_D', 'x_A']
-----
cItems1
['x_B', 'x_C', 'x_E']
-----
cVar0
1.0000000000000013
-----
cVar1
0.55527382075709
-----
alpha
0.3570264048338372
-----
cItems0
['x_D']
-----
cItems1
['x_A']
-----
cVar0
1.0000000000000013
-----
cVar1
1.0000000000000013
-----
alpha
0.5
-----
cItems0
['x_B']
-----
cItems1
['x_C', 'x_E']
-----
cVar0
1.0000000000000013
-----
cVar1
0.9999999999999964
-----
alpha
0.4999999999999988
-----
cItems0
['x_C']
-----
cItems1
['x_E']
-----
cVar0
0.9999999999999964
-----
cVar1
0.9999999999999966
-----
alpha
0.5
-----


C:\Users\Kyle\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: ClusterWarning: scipy.cluster: The symmetric non-negative hollow observation matrix looks suspiciously like an uncondensed distance matrix


What happened?

Well, at the first bisection, we have the 5 assets separated as {A, D} and {B, C, E}.  

This is different than Mystery 1.  So we're already off to a different start.  {A, B, D} are correlated to each other, but that cluster is destroyed at the first bisection.  

Unlike Mystery 1, the sets {A, D} and {B, C, E} do not receive equal weights.  {A, D} gets "alpha" (weight) equal to 0.357, and {B, C, E} gets 64.3% capital (1 minus "alpha") to start the race to the bottom of the tree.

When we bisect {B, C, E}, we create the subsets {B} & {C, E}.  Each gets equal allocation (half of 64.3%).

That's why B ends up with a whopping 32% weight for epsilon = 0, but a drastically different weight of 25% for epsilon = 0.1; the smallest perturbation can affect outputs.  

And since HRP doesn't really account for correlations, despite JoPM's claims, we get ridiculous results like this.